# 03 - Vector Search Homework

## Q1. Getting the embeddings model

In [25]:
from tqdm.auto import tqdm
import numpy as np

In [1]:
model_name = 'multi-qa-distilbert-cos-v1'

In [2]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)

/home/onwunalu/.pyenv/versions/llm-zoomcamp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
user_question = "I just discovered the course. Can I still join it?"


In [5]:
user_question_vector = embedding_model.encode(user_question)
user_question_vector[0]

0.078222655

### Q1b: Prepare the documents

In [6]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [8]:
# filter the document
documents_filtered = [doc for doc in documents if doc['course'] == 'machine-learning-zoomcamp']

In [9]:
len(documents_filtered)

375

In [ ]:
"""
Create a list embeddings
Iterate over each document
qa_text = f'{question} {text}'
compute the embedding for qa_text, append to embeddings
At the end, let X = np.array(embeddings) (import numpy as np)
"""

In [26]:
embeddings = []
for doc in tqdm(documents_filtered):
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    embeddings.append(embedding_model.encode(qa_text))


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 375/375 [00:24<00:00, 15.10it/s]


In [29]:
X = np.array(embeddings)
X.shape

(375, 768)

# Q3. Search

In [30]:
scores = X.dot(user_question_vector)

In [31]:
max(scores)

0.6506571

## Vector search

In [33]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(user_question_vector, num_results=5)

[{'text': 'You can find the latest and up-to-date deadlines here: https://docs.google.com/spreadsheets/d/e/2PACX-1vQACMLuutV5rvXg5qICuJGL-yZqIV0FBD84CxPdC5eZHf8TfzB-CJT_3Mo7U7oGVTXmSihPgQxuuoku/pubhtml\nAlso, take note of Announcements from @Au-Tomator for any extensions or other news. Or, the form may also show the updated deadline, if Instructor(s) has updated it.',
  'section': 'General course-related questions',
  'question': 'Homework - What are homework and project deadlines?',
  'course': 'data-engineering-zoomcamp',
  'id': 'a1daf537'},
 {'text': 'After you submit your homework it will be graded based on the amount of questions in a particular homework. You can see how many points you have right on the page of the homework up top. Additionally in the leaderboard you will find the sum of all points you’ve earned - points for Homeworks, FAQs and Learning in Public. If homework is clear, others work as follows: if you submit something to FAQ, you get one point, for each learning i

# Q4. Hit-rate for our search engine

In [34]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [46]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

def new_vector_search_wrapper(doc):
    question = doc['question']
    query_vector = embedding_model.encode(question)
    return search_engine.search(query_vector, num_results=5)
    

In [47]:
evaluate(ground_truth, new_vector_search_wrapper)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [03:38<00:00,  8.38it/s]


{'hit_rate': 0.0, 'mrr': 0.0}

# Q5. Indexing with Elasticsearch

In [49]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [48]:
# add the computed vectors to the document

In [51]:
for i, doc in tqdm(enumerate(documents_filtered)):
    doc['question_text_vector'] = X[i,:]

375it [00:00, 552269.66it/s]


In [52]:
# index the documents with the vectors
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:19<00:00, 48.67it/s]


In [67]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

def question_text_vector_knn(doc):
    question = doc['question']

    v_q = embedding_model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course='machine-learning-zoomcamp')

In [69]:
# Get the id of the top document returned for the query in Question 1
results = elastic_search_knn('question_text_vector', user_question_vector, course='machine-learning-zoomcamp')
results[0]['id']

'ee58a693'

In [ ]:
# id of the top document returned is: 'id': 'ee58a693'

# Q6. Hit-rate for Elasticsearch

In [68]:
evaluate(ground_truth, question_text_vector_knn)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1830/1830 [00:49<00:00, 36.73it/s]


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}